In [14]:
import os, sys, re
import numpy as np
import pandas as pd
from pathlib import Path
from functools import reduce
from pandas import HDFStore,DataFrame
# define the current path (notebooks in lab_utils)
currpath = os.getcwd()
labutilspath = str(Path(currpath).parents[1])
sys.path.append(labutilspath)

# import the autoscan routines
from autoscan import autoscan

# define paths
basepath = '/home/urlab/sandbox/data/autoscan/'

# set the asdatapath accordingly (where is the atuoscan data inside basedatapath?)
asdatapath   = 'new_measurements'

# set datapath
datapath = os.path.join(basepath, asdatapath)

pp = autoscan.postprocess(labutilspath=labutilspath)
pp.debug = False

In [15]:
# load the summary dataframe with all measurement
summary = pd.read_csv(os.path.join(datapath,'summary.csv'))
# remove ztop and zbottom
t = summary['relroot'].apply(lambda x: len(re.findall(r'ztop|zbottom', x))==0)
summary = summary.loc[t,:].copy()

sample_info = pp._subset_info(summary, instance = instance, sample_tag = tag)

In [22]:

# dfs = []
# for s in sample_info.itertuples(index=False):
#     relpath = getattr(s, 'relroot')
#     probe   = getattr(s, 'probe')
#     data = pp.read_data(os.path.join(datapath, relpath), probe = probe)
#     h = pp.probe_settings[probe]['h']
#     data = data.iloc[:,:h]
#     if probe == 'vel':
#         for x in data.angle.unique():
#             dfs.append(data.loc[data['angle'] == x,:].copy().drop(columns = 'angle'))
#     else:
#         dfs.append(data)

# df_merged = reduce(lambda left,right: pd.merge(left,right,on=['x','y'], how ='inner'), dfs)
# df_merged = pp._enforce_float(df_merged)
# df_merged['tag'] = tag
# df_merged['instance'] = instance
summary

,sample_tag,subsample_tag,sample_code,sample_family,probe,side,instance,fname,relroot
0,wsg_002,plugs,sg,sandstone,ftir,3.0,before,ftir_before_3.csv,wsg_002/subsamples/plugs/processed/ftir_before...
1,wsg_002,plugs,sg,sandstone,ftir,2.0,before,ftir_before_2.csv,wsg_002/subsamples/plugs/processed/ftir_before...
2,wsg_002,plugs,sg,sandstone,perm,NaN,before,perm_before.csv,wsg_002/subsamples/plugs/processed/perm_before...
3,wsg_002,plugs,sg,sandstone,ftir,6.0,before,ftir_before_6.csv,wsg_002/subsamples/plugs/processed/ftir_before...
4,wsg_002,plugs,sg,sandstone,ftir,1.0,before,ftir_before_1.csv,wsg_002/subsamples/plugs/processed/ftir_before...
...,...,...,...,...,...,...,...,...,...
63,wsg_003,s4,sg,sandstone,vel,NaN,before,vel_before.csv,wsg_003/subsamples/s4/processed/vel_before.csv
64,wsg_003,s3,sg,sandstone,perm,NaN,before,perm_before.csv,wsg_003/subsamples/s3/processed/perm_before.csv
65,wsg_003,s3,sg,sandstone,vel,NaN,before,vel_before.csv,wsg_003/subsamples/s3/processed/vel_before.csv
66,wsg_003,s2,sg,sandstone,perm,NaN,before,perm_before.csv,wsg_003/subsamples/s2/processed/perm_before.csv


In [46]:
save = False
fullset = []
problem = {}
problem_merged = []
ns = 0
for instace in ['before', 'after']:
    for tag in summary['sample_tag'].unique():
        s1 = pp._subset_info(summary, instance = instance, sample_tag = tag)
        if len(s1)>0:
            family = s1['sample_family'].unique()[0]
            code   = s1['sample_code'].unique()[0]
            for subtag in s1['subsample_tag'].unique():
                s2 = pp._subset_info(s1, subsample_tag = subtag, sample_code = code, sample_family = family)
                for kside, side in enumerate(s2['side'].unique()):
                    s3 = pp._subset_info(s2, side = side)
                    probes  = s3['probe'].unique()
                    nprobes = len(probes)
                    if nprobes>=1 and ('ftir' in probes):
                        dfs = []
                        for meas in s3.itertuples(index=False):
                            relpath  = getattr(meas, 'relroot')
                            filepath = os.path.join(datapath, relpath)
                            probe    = getattr(meas, 'probe')
                            
                            data     = pp.read_data(filepath, probe = probe)
                            
                            h    = pp.probe_settings[probe]['h']
                            data = data.iloc[:,:h]

                            if probe == 'vel':
                                for k,x in enumerate(data.angle.unique()):
                                    colnames = ['vp_'+str(k), 'vs_'+str(k)]
                                    data_temp = data.loc[data['angle'] == x,:].copy().drop(columns = 'angle')
                                    data_temp.columns = ['x','y'] + colnames
                                    dfs.append(data_temp)
                                    del data_temp
                            else:
                                dfs.append(data)

                        df_merged = reduce(lambda left,right: pd.merge(left,right,on=['x','y'], how ='inner'), dfs)
                        df_merged = pp._enforce_float(df_merged)
                        if df_merged.shape[0]>0 and len(df_merged.iloc[:,1754:].columns.values)>=0:
                            ns += df_merged.shape[0]
                            df_merged['family']   = family
                            df_merged['code']     = code
                            df_merged['tag']      = tag
                            df_merged['subtag']   = subtag
                            df_merged['instance'] = instance
                            df_merged['side']     = kside
                            
                            if save:
                                outpath = pp._set_outpath(datapath,tag)
                                outname = pp._set_outfilename(tag, subtag, side, instance, 'fullset')
                                pp.save_data(df_merged, savepath = outpath,  savename = outname)
                        
                            fullset.append(df_merged.iloc[:,2:])

In [47]:
ns
compiledinfocols = ['family', 'code', 'tag', 'subtag', 'side', 'instance']
compileddatacols = pp.probe_settings['ftir']['names'][2:] + ['perm', 'vp_0', 'vs_0']  
compiledfullcols = compiledinfocols + compileddatacols

dfc = pd.concat(fullset, sort = False, axis = 0, ignore_index=True, join = 'outer')
dfc = dfc.loc[:,compiledfullcols].copy()
dfc.loc[:,compileddatacols] = dfc.loc[:,compileddatacols].apply(pd.to_numeric,errors='coerce').copy()

In [50]:
dfc.subtag.unique()
# savefile = os.path.join(datapath, 'as_dataset.h5')
# dfc.to_hdf(savefile, key = 'data', mode = 'w')

array(['plugs', 's1', 's6', 's5', 's4', 's2', 's7', 's3'], dtype=object)

In [52]:
import deepdish as dp
dp.io.save(os.path.join(datapath, 'compiled.h5'), dfc)
# dp.io.save(os.path.join(datapath, 'data.h5'), dfc.loc[:, compileddatacols])
dp.io.save(os.path.join(datapath, 'descriptions.h5'), dfc.loc[:, compiledinfocols])

In [53]:
# dp.io.save(os.path.join(datapath, 'descriptions.h5'), dfc.loc[:, compiledinfocols])

In [54]:
# dp.io.save(os.path.join(datapath, 'compiled.h5'), dfc)